<a href="https://colab.research.google.com/github/tahereh-fahi/AI-ML-projects/blob/main/AnalyzingReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Automatic Analyzing review (Colab Notebook)

**Status:** Private research notebook for academic evaluation.

**Do not publish or make this notebook public.** MIT rules restrict sharing of project code.
Grant view access *on request* to instructors/reviewers only.

**Notebook generated:** 2025-08-24 18:02

In [ ]:
# Core libraries
import os
import sys
import csv
import re
import json
import random
import math
import string
import shutil

# Data / math
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Collections / utilities
from collections import Counter, defaultdict, OrderedDict, deque
from pathlib import Path

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import (
    confusion_matrix, classification_report,
    accuracy_score, roc_auc_score
)
from sklearn.preprocessing import label_binarize, normalize

# NLTK (if you use text preprocessing)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Optional: tqdm progress bar, requests/bs4 for scraping, etc.
# from tqdm import tqdm
# import requests
# from bs4 import BeautifulSoup

# Compatibility flag for old code
PYTHON3 = sys.version_info[0] >= 3


## 0) Environment & file setup
- This notebook auto-downloads the dataset from a **shared Google Drive folder link** into `DATA_DIR` using `gdown --folder`.
- If any files are still missing, it falls back to **manual upload**.
- Keep the notebook **Restricted** (Share → General access: *Restricted*).

In [ ]:
# --- RUN ME FIRST: download & prepare dataset(s) from GitHub Releases -------
import os, subprocess, hashlib, pathlib, zipfile, tarfile

os.makedirs("data", exist_ok=True)

def download(url: str, out_path: str):
    print(f"Downloading {url} -> {out_path}")
    subprocess.run(
        ["curl", "-L", "--fail", "--retry", "3", "--retry-delay", "3", "-o", out_path, url],
        check=True
    )

def sha256sum(path: str) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1<<20), b""):
            h.update(chunk)
    return h.hexdigest()

def verify(path: str, expected_sha256: str | None):
    if not expected_sha256:
        print("Skipping checksum verification.")
        return
    actual = sha256sum(path)
    if actual.lower() != expected_sha256.lower():
        raise RuntimeError(f"Checksum mismatch for {path}\nexpected: {expected_sha256}\nactual:   {actual}")
    print("Checksum OK:", actual)

def extract_if_archive(path: str, dest_dir="data"):
    p = pathlib.Path(path)
    name = p.name.lower()
    if name.endswith(".zip"):
        with zipfile.ZipFile(path) as z:
            z.extractall(dest_dir)
        print("Unzipped to", dest_dir)
    elif name.endswith(".tar.gz") or name.endswith(".tgz"):
        with tarfile.open(path, "r:gz") as t:
            t.extractall(dest_dir)
        print("Extracted tar.gz to", dest_dir)
    else:
        print("Not an archive; nothing to extract.")

# === FILL THESE THREE LINES WITH YOUR RELEASE INFO ==========================
URL = "https://github.com/tahereh-fahi/Data/releases/download/v1.0.0/reviewAmazonData.zip"
OUT = "data/my_dataset_v1.zip"
SHA256 = "3ad20851fd564e93baad7d878e9d90fab314698206f5c228e54bf0bca8940135"  # or set to None to skip
# ============================================================================
download(URL, OUT)
verify(OUT, SHA256)
extract_if_archive(OUT, "data")

print("Ready. Your files are in ./data")


Checksum OK: 3ad20851fd564e93baad7d878e9d90fab314698206f5c228e54bf0bca8940135
Unzipped to data
Ready. Your files are in ./data


## 1a) Utilities (data I/O, tuning helpers, feature inspection)

In [ ]:
def load_data(path_data, extras=False):
    """
    Returns a list of dict with keys:
    * sentiment: +1 or -1 if the review was positive or negative, respectively
    * text: the text of the review

    Additionally, if the `extras` argument is True, each dict will also include the
    following information:
    * productId: a string that uniquely identifies each product
    * userId: a string that uniquely identifies each user
    * summary: the title of the review
    * helpfulY: the number of users who thought this review was helpful
    * helpfulN: the number of users who thought this review was NOT helpful
    """

    global PYTHON3

    basic_fields = {'sentiment', 'text'}
    numeric_fields = {'sentiment', 'helpfulY', 'helpfulN'}

    data = []
    if PYTHON3:
        f_data = open(path_data, encoding="latin1")
    else:
        f_data = open(path_data)

    for datum in csv.DictReader(f_data, delimiter='\t'):
        for field in list(datum.keys()):
            if not extras and field not in basic_fields:
                del datum[field]
            elif field in numeric_fields and datum[field]:
                datum[field] = int(datum[field])

        data.append(datum)

    f_data.close()

    return data

def write_predictions(path_submit_data, preds):
    if PYTHON3:
        f_data = open(path_submit_data, encoding="latin1")
    else:
        f_data = open(path_submit_data, 'rb')

    reader = csv.DictReader(f_data, delimiter='\t')
    data = list(reader)

    assert len(preds) == len(data), \
           'Expected {} predictions but {} were given.'.format(len(data), len(preds))

    for pred, datum in zip(preds.astype(int), data):
        assert pred == 1 or pred == -1, 'Invalid prediction: {}.'.format(pred)
        datum['sentiment'] = pred
    f_data.close()

    if PYTHON3:
        f_out = open(path_submit_data, 'w')
    else:
        f_out = open(path_submit_data, 'wb')

    writer = csv.DictWriter(f_out, delimiter='\t', fieldnames=reader.fieldnames)
    writer.writeheader()
    for datum in data:
        writer.writerow(datum)
    f_out.close()

def tune(train_fn, param_vals, train_feats, train_labels, val_feats, val_labels):
    train_accs = np.ndarray(len(param_vals))
    val_accs = np.ndarray(len(param_vals))

    for i, val in enumerate(param_vals):
        theta, theta_0 = train_fn(train_feats, train_labels, val)

        train_preds = classify(train_feats, theta, theta_0)
        train_accs[i] = accuracy(train_preds, train_labels)

        val_preds = classify(val_feats, theta, theta_0)
        val_accs[i] = accuracy(val_preds, val_labels)

    return train_accs, val_accs

def tune_perceptron(*args):
    return tune(perceptron, *args)

def tune_avg_perceptron(*args):
    return tune(average_perceptron, *args)

def tune_pegasos_T(best_L, *args):
    def train_fn(features, labels, T):
        return pegasos(features, labels, T, best_L)
    return tune(train_fn, *args)

def tune_pegasos_L(best_T, *args):
    def train_fn(features, labels, L):
        return pegasos(features, labels, best_T, L)
    return tune(train_fn, *args)

def most_explanatory_word(theta, wordlist):
    """Returns the word associated with the bag-of-words feature having largest weight."""
    return [word for (theta_i, word) in sorted(zip(theta, wordlist))[::-1]]

## 2) Core algorithms & feature builders

In [ ]:
def get_order(n_samples):
    try:
        with open(str(n_samples) + '.txt') as fp:
            line = fp.readline()
            return list(map(int, line.split(',')))
    except FileNotFoundError:
        random.seed(1)
        indices = list(range(n_samples))
        random.shuffle(indices)
        return indices

def hinge_loss_single(feature_vector, label, theta, theta_0):
    """
    Finds the hinge loss on a single data point given specific classification
    parameters.

    Args:
        feature_vector - A numpy array describing the given data point.
        label - A real valued number, the correct classification of the data
            point.
        theta - A numpy array describing the linear classifier.
        theta_0 - A real valued number representing the offset parameter.


    Returns: A real number representing the hinge loss associated with the
    given data point and parameters.
    """
    # Your code
    z = (np.dot(theta,feature_vector)+theta_0)*label
    if z >=1:
        return 0
    else:
        return (1-z)

def hinge_loss_full(feature_matrix, labels, theta, theta_0):
    """
    Finds the total hinge loss on a set of data given specific classification
    parameters.

    Args:
        feature_matrix - A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        theta - A numpy array describing the linear classifier.
        theta_0 - A real valued number representing the offset parameter.


    Returns: A real number representing the hinge loss associated with the
    given dataset and parameters. This number should be the average hinge
    loss across all of the points in the feature matrix.
    """
    # Your code
    Z = (np.dot(feature_matrix, theta)+theta_0)*labels
    hing_loss = 0
    for z in Z:
        if z<1:
            hing_loss += (1-z)

    hing_loss_totall = hing_loss / np.shape(feature_matrix)[0]

    return hing_loss_totall

def perceptron_single_step_update(
        feature_vector,
        label,
        current_theta,
        current_theta_0):
    """
    Properly updates the classification parameter, theta and theta_0, on a
    single step of the perceptron algorithm.

    Args:
        feature_vector - A numpy array describing a single data point.
        label - The correct classification of the feature vector.
        current_theta - The current theta being used by the perceptron
            algorithm before this update.
        current_theta_0 - The current theta_0 being used by the perceptron
            algorithm before this update.

    Returns: A tuple where the first element is a numpy array with the value of
    theta after the current update has completed and the second element is a
    real valued number with the value of theta_0 after the current updated has
    completed.
    """
    # Your code here
    if label * (np.dot(current_theta , feature_vector) + current_theta_0) <= 0:
        theta = current_theta + label*feature_vector
        theta_0 = current_theta_0 + label

    else :
        theta = current_theta
        theta_0 = current_theta_0

    return (theta, theta_0)

def perceptron(feature_matrix, labels, T):
    """
    Runs the full perceptron algorithm on a given set of data. Runs T
    iterations through the data set, there is no need to worry about
    stopping early.

    NOTE: Please use the previously implemented functions when applicable.
    Do not copy paste code from previous parts.

    NOTE: Iterate the data matrix by the orders returned by get_order(feature_matrix.shape[0])

    Args:
        feature_matrix -  A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        T - An integer indicating how many times the perceptron algorithm
            should iterate through the feature matrix.

    Returns: A tuple where the first element is a numpy array with the value of
    theta, the linear classification parameter, after T iterations through the
    feature matrix and the second element is a real number with the value of
    theta_0, the offset classification parameter, after T iterations through
    the feature matrix.
    """

    # Your code here
    theta = np.zeros(feature_matrix.shape[1])
    theta_0 = 0
    for t in range(T):
        for i in get_order(feature_matrix.shape[0]):
            # Your code here
            theta , theta_0 = perceptron_single_step_update(feature_matrix[i,:] ,labels[i],theta,theta_0)

    return (theta, theta_0)

def average_perceptron(feature_matrix, labels, T):
    """
    Runs the average perceptron algorithm on a given set of data. Runs T
    iterations through the data set, there is no need to worry about
    stopping early.

    NOTE: Please use the previously implemented functions when applicable.
    Do not copy paste code from previous parts.

    NOTE: Iterate the data matrix by the orders returned by get_order(feature_matrix.shape[0])


    Args:
        feature_matrix -  A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        T - An integer indicating how many times the perceptron algorithm
            should iterate through the feature matrix.

    Returns: A tuple where the first element is a numpy array with the value of
    the average theta, the linear classification parameter, found after T
    iterations through the feature matrix and the second element is a real
    number with the value of the average theta_0, the offset classification
    parameter, found after T iterations through the feature matrix.

    Hint: It is difficult to keep a running average; however, it is simple to
    find a sum and divide.
    """
    # Your code here
    n = feature_matrix.shape[0]
    d = feature_matrix.shape[1]

    theta = np.zeros(d)
    theta_0 = 0

    theta_sum = np.zeros(d)
    theta_0_sum = 0

    for t in range(T):
        for i in get_order(feature_matrix.shape[0]):
            # Your code here
            theta , theta_0 = perceptron_single_step_update(feature_matrix[i,:] ,labels[i],theta,theta_0)
            theta_sum += theta
            theta_0_sum += theta_0

    theta_avg = theta_sum/(n*T)
    theta_0_avg = theta_0_sum/(n*T)

    return (theta_avg, theta_0_avg)

def pegasos_single_step_update(
        feature_vector,
        label,
        L,
        eta,
        current_theta,
        current_theta_0):
    """
    Properly updates the classification parameter, theta and theta_0, on a
    single step of the Pegasos algorithm

    Args:
        feature_vector - A numpy array describing a single data point.
        label - The correct classification of the feature vector.
        L - The lamba value being used to update the parameters.
        eta - Learning rate to update parameters.
        current_theta - The current theta being used by the Pegasos
            algorithm before this update.
        current_theta_0 - The current theta_0 being used by the
            Pegasos algorithm before this update.

    Returns: A tuple where the first element is a numpy array with the value of
    theta after the current update has completed and the second element is a
    real valued number with the value of theta_0 after the current updated has
    completed.
    """
    # Your code here
    if label * (np.dot(current_theta , feature_vector) + current_theta_0) <= 1:

        theta = (1- eta*L)* current_theta + eta *label*feature_vector
        theta_0 = current_theta_0 + eta *label

    else :

        theta = (1- eta*L)* current_theta
        theta_0 = current_theta_0

    return (theta , theta_0)

def pegasos(feature_matrix, labels, T, L):
    """
    Runs the Pegasos algorithm on a given set of data. Runs T
    iterations through the data set, there is no need to worry about
    stopping early.

    For each update, set learning rate = 1/sqrt(t),
    where t is a counter for the number of updates performed so far (between 1
    and nT inclusive).

    NOTE: Please use the previously implemented functions when applicable.
    Do not copy paste code from previous parts.

    Args:
        feature_matrix - A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        T - An integer indicating how many times the algorithm
            should iterate through the feature matrix.
        L - The lamba value being used to update the Pegasos
            algorithm parameters.

    Returns: A tuple where the first element is a numpy array with the value of
    the theta, the linear classification parameter, found after T
    iterations through the feature matrix and the second element is a real
    number with the value of the theta_0, the offset classification
    parameter, found after T iterations through the feature matrix.
    """
    # Your code here
    d = feature_matrix.shape[1]

    theta = np.zeros(d)
    theta_0 = 0

    t = 1

    for tee in range(T):


        for i in get_order(feature_matrix.shape[0]):

            eta = 1/(np.sqrt(t))
            theta , theta_0 = pegasos_single_step_update(feature_matrix[i,:] ,labels[i], L, eta, theta, theta_0)
            t += 1

    return (theta, theta_0)

def classify(feature_matrix, theta, theta_0):
    """
    A classification function that uses theta and theta_0 to classify a set of
    data points.

    Args:
        feature_matrix - A numpy matrix describing the given data. Each row
            represents a single data point.
                theta - A numpy array describing the linear classifier.
        theta - A numpy array describing the linear classifier.
        theta_0 - A real valued number representing the offset parameter.

    Returns: A numpy array of 1s and -1s where the kth element of the array is
    the predicted classification of the kth row of the feature matrix using the
    given theta and theta_0. If a prediction is GREATER THAN zero, it should
    be considered a positive classification.
    """

    yhat = np.dot(feature_matrix,theta)+theta_0

    A = []
    for yh in yhat:
        if yh > 0:
            A.append(1)
        else:
            A.append(-1)

    return np.array(A)

def classifier_accuracy(
        classifier,
        train_feature_matrix,
        val_feature_matrix,
        train_labels,
        val_labels,
        **kwargs):
    """
    Trains a linear classifier and computes accuracy.
    The classifier is trained on the train data. The classifier's
    accuracy on the train and validation data is then returned.

    Args:
        classifier - A classifier function that takes arguments
            (feature matrix, labels, **kwargs) and returns (theta, theta_0)
        train_feature_matrix - A numpy matrix describing the training
            data. Each row represents a single data point.
        val_feature_matrix - A numpy matrix describing the validation
            data. Each row represents a single data point.
        train_labels - A numpy array where the kth element of the array
            is the correct classification of the kth row of the training
            feature matrix.
        val_labels - A numpy array where the kth element of the array
            is the correct classification of the kth row of the validation
            feature matrix.
        **kwargs - Additional named arguments to pass to the classifier
            (e.g. T or L)

    Returns: A tuple in which the first element is the (scalar) accuracy of the
    trained classifier on the training data and the second element is the
    accuracy of the trained classifier on the validation data.
    """
    # train model
    theta , theta_0 = classifier(train_feature_matrix, train_labels,**kwargs)

    # train accuracy

    yhat_train = np.dot(train_feature_matrix,theta)+theta_0
    A1 = []
    for yht in yhat_train:
        if yht > 0:
            A1.append(1)
        else:
            A1.append(-1)

    train_labels_pred = np.array(A1)

    train_accu = accuracy(train_labels_pred,train_labels)

    # validation accuracy

    yhat_val = np.dot(val_feature_matrix,theta)+theta_0
    A2 = []
    for yht in yhat_val:
        if yht > 0:
            A2.append(1)
        else:
            A2.append(-1)

    val_labels_pred = np.array(A2)

    val_accu = accuracy(val_labels_pred,val_labels)

    return (train_accu, val_accu)

def extract_words(input_string):
    """
    Helper function for bag_of_words()
    Inputs a text string
    Returns a list of lowercase words in the string.
    Punctuation and digits are separated out into their own words.
    """
    for c in string.punctuation + string.digits:
        input_string = input_string.replace(c, ' ' + c + ' ')

    return input_string.lower().split()

def bag_of_words(texts):
    """
    Inputs a list of string reviews
    Returns a dictionary of unique unigrams occurring over the input

    Feel free to change this code as guided by Problem 9
    """
    # Your code here
    dictionary = {} # maps word to unique index
    for text in texts:
        word_list = extract_words(text)
        for word in word_list:
            if word not in dictionary:
                dictionary[word] = len(dictionary)
    return dictionary

def extract_bow_feature_vectors(reviews, dictionary):
    """
    Inputs a list of string reviews
    Inputs the dictionary of words as given by bag_of_words
    Returns the bag-of-words feature matrix representation of the data.
    The returned matrix is of shape (n, m), where n is the number of reviews
    and m the total number of entries in the dictionary.

    Feel free to change this code as guided by Problem 9
    """
    # Your code here

    num_reviews = len(reviews)
    feature_matrix = np.zeros([num_reviews, len(dictionary)])

    for i, text in enumerate(reviews):
        word_list = extract_words(text)
        for word in word_list:
            if word in dictionary:
                feature_matrix[i, dictionary[word]] = 1
    return feature_matrix

def accuracy(preds, targets):
    """
    Given length-N vectors containing predicted and target labels,
    returns the percentage and number of correct predictions.
    """
    return (preds == targets).mean()

def classifier_accuracy_sanaz(
        classifier,
        train_feature_matrix,
        val_feature_matrix,
        train_labels,
        val_labels,
        **kwargs):
    """
    Trains a linear classifier and computes accuracy.
    The classifier is trained on the train data. The classifier's
    accuracy on the train and validation data is then returned.

    Args:
        classifier - A classifier function that takes arguments
            (feature matrix, labels, **kwargs) and returns (theta, theta_0)
        train_feature_matrix - A numpy matrix describing the training
            data. Each row represents a single data point.
        val_feature_matrix - A numpy matrix describing the validation
            data. Each row represents a single data point.
        train_labels - A numpy array where the kth element of the array
            is the correct classification of the kth row of the training
            feature matrix.
        val_labels - A numpy array where the kth element of the array
            is the correct classification of the kth row of the validation
            feature matrix.
        **kwargs - Additional named arguments to pass to the classifier
            (e.g. T or L)

    Returns: A tuple in which the first element is the (scalar) accuracy of the
    trained classifier on the training data and the second element is the
    accuracy of the trained classifier on the validation data.
    """
    # train model
    theta , theta_0 = classifier(train_feature_matrix, train_labels,**kwargs)


    return (theta, theta_0)

def bag_of_words_sanaz(texts):
    """
    Inputs a list of string reviews
    Returns a dictionary of unique unigrams occurring over the input

    Feel free to change this code as guided by Problem 9
    """
    stopwords_list = load_data(os.path.join("/content/data", "reviews_train.tsv"))


    # Your code here
    dictionary = {} # maps word to unique index
    for text in texts:
        word_list = extract_words(text)
        for word in word_list:
            if (word not in dictionary) and [word] not in stopwords_list :
                dictionary[word] = len(dictionary)
    return dictionary

def extract_bow_feature_vectors_sanaz(reviews, dictionary):
    """
    Inputs a list of string reviews
    Inputs the dictionary of words as given by bag_of_words
    Returns the bag-of-words feature matrix representation of the data.
    The returned matrix is of shape (n, m), where n is the number of reviews
    and m the total number of entries in the dictionary.

    Feel free to change this code as guided by Problem 9
    """
    # Your code here

    num_reviews = len(reviews)
    feature_matrix = np.zeros([num_reviews, len(dictionary)])

    for i, text in enumerate(reviews):
        word_list = extract_words(text)
        for word in word_list:
            if word in dictionary:
                feature_matrix[i, dictionary[word]] = word_list.count(word)
    return feature_matrix

In [ ]:
train_data = load_data(os.path.join("/content/data", "reviews_train.tsv"))
val_data   = load_data(os.path.join("/content/data", "reviews_val.tsv"))
test_data  = load_data(os.path.join("/content/data", "reviews_test.tsv"))

train_texts, train_labels = zip(*((s["text"], s["sentiment"]) for s in train_data))
val_texts,   val_labels   = zip(*((s["text"],   s["sentiment"]) for s in val_data))
test_texts,  test_labels  = zip(*((s["text"],   s["sentiment"]) for s in test_data))

import numpy as np, pandas as pd
train_labels = np.array(train_labels); val_labels = np.array(val_labels); test_labels = np.array(test_labels)

print(f"Train: {len(train_texts)} | Val: {len(val_texts)} | Test: {len(test_texts)}")
print("Label balance (train):", np.unique(train_labels, return_counts=True))
print("Label balance (val):  ", np.unique(val_labels, return_counts=True))

pd.DataFrame(train_data[:5])

Train: 4000 | Val: 500 | Test: 500
Label balance (train): (array([-1,  1]), array([2030, 1970]))
Label balance (val):   (array([-1,  1]), array([259, 241]))


,sentiment,text
0,-1,The chips are okay Not near as flavorful as th...
1,-1,"I had high hopes for this, but it was bad. Re..."
2,-1,I guess it's only one can since there is nothi...
3,-1,"""Oatmeal Squares"" is in about the largest prin..."
4,1,"I really enjoyed this flavor, this has a very ..."


## 3) Bag‑of‑Words dictionaries and features
We build classic unigram features with two variants:
- **All tokens**
- **Stopwords removed** (using `stopwords.txt`)

In [ ]:

# With all tokens
dictionary_all = bag_of_words(train_texts)
Xtr_all = extract_bow_feature_vectors(train_texts, dictionary_all)
Xva_all = extract_bow_feature_vectors(val_texts,   dictionary_all)
Xte_all = extract_bow_feature_vectors(test_texts,  dictionary_all)
print("All-tokens dictionary size:", len(dictionary_all), "| Xtr shape:", Xtr_all.shape)

# With stopwords removed
dictionary_sw = bag_of_words_sanaz(train_texts)
Xtr_sw = extract_bow_feature_vectors_sanaz(train_texts, dictionary_sw)
Xva_sw = extract_bow_feature_vectors_sanaz(val_texts,   dictionary_sw)
Xte_sw = extract_bow_feature_vectors_sanaz(test_texts,  dictionary_sw)
print("Stopword‑filtered dictionary size:", len(dictionary_sw), "| Xtr shape:", Xtr_sw.shape)


All-tokens dictionary size: 13234 | Xtr shape: (4000, 13234)
Stopword‑filtered dictionary size: 13234 | Xtr shape: (4000, 13234)


## 4) Train & evaluate: Perceptron / Avg Perceptron / Pegasos

In [ ]:

T = 10
L = 0.01

print("=== Using stopword‑filtered features ===")
pct_tr, pct_va = classifier_accuracy(perceptron, Xtr_sw, Xva_sw, train_labels, val_labels, T=T)
print(f"Perceptron — train: {pct_tr:.4f} | val: {pct_va:.4f}")

ap_tr, ap_va = classifier_accuracy(average_perceptron, Xtr_sw, Xva_sw, train_labels, val_labels, T=T)
print(f"Avg Perceptron — train: {ap_tr:.4f} | val: {ap_va:.4f}")

peg_tr, peg_va = classifier_accuracy(pegasos, Xtr_sw, Xva_sw, train_labels, val_labels, T=T, L=L)
print(f"Pegasos — train: {peg_tr:.4f} | val: {peg_va:.4f}")


=== Using stopword‑filtered features ===
Perceptron — train: 0.9123 | val: 0.7860
Avg Perceptron — train: 0.8812 | val: 0.7940
Pegasos — train: 0.8768 | val: 0.8040


## 5) Hyperparameter tuning (small sweep)

In [ ]:
T_vals = [1, 3, 5, 10, 15, 25]
L_vals = [1e-3, 3e-3, 1e-2, 3e-2, 1e-1]

# Tune Pegasos T first (fixing lambda), then lambda (fixing T)
tr, va = tune_pegasos_T(L, T_vals, Xtr_sw, train_labels, Xva_sw, val_labels)
best_T = T_vals[int(np.argmax(va))]
print("Tuning T:", dict(zip(T_vals, va)), "| best T:", best_T)

tr, va = tune_pegasos_L(best_T, L_vals, Xtr_sw, train_labels, Xva_sw, val_labels)
best_L = L_vals[int(np.argmax(va))]
print("Tuning lambda:", dict(zip(L_vals, va)), "| best lambda:", best_L)

Tuning T: {1: np.float64(0.708), 3: np.float64(0.742), 5: np.float64(0.744), 10: np.float64(0.804), 15: np.float64(0.776), 25: np.float64(0.78)} | best T: 10
Tuning lambda: {0.001: np.float64(0.784), 0.003: np.float64(0.778), 0.01: np.float64(0.804), 0.03: np.float64(0.746), 0.1: np.float64(0.688)} | best lambda: 0.01


## 6) Final model & predictions

In [ ]:
final_theta, final_theta0 = pegasos(Xtr_sw, train_labels, T=best_T, L=best_L)
test_preds = classify(Xte_sw, final_theta, final_theta0)
test_acc = accuracy(test_preds, test_labels)
print(f"Final test accuracy (Pegasos, stopword‑filtered): {test_acc:.4f}")

# Write predictions for the competition/submit set
submit_data = load_data(os.path.join("/content/data", "reviews_submit.tsv"))
submit_texts = [s["text"] for s in submit_data]
Xsub = extract_bow_feature_vectors_sanaz(submit_texts, dictionary_sw)
submit_preds = classify(Xsub, final_theta, final_theta0)

out_path = os.path.join("/content/data", "predictions_submit.tsv")
with open(out_path, "w", encoding="utf-8") as f:
    for y in submit_preds:
        f.write(f"{y}\n")
print("Wrote:", out_path)

Final test accuracy (Pegasos, stopword‑filtered): 0.7740
Wrote: /content/data/predictions_submit.tsv


## 7) Most explanatory words

In [ ]:

# Map weights back to words
inv_vocab = {idx: w for w, idx in dictionary_sw.items()}
wordlist = [inv_vocab[i] for i in range(len(inv_vocab))]

# Sort words by weight (descending)
sorted_pairs = sorted(zip(final_theta, wordlist), key=lambda x: x[0], reverse=True)
top_pos = [w for _, w in sorted_pairs[:25]]
top_neg = [w for _, w in sorted_pairs[-25:]]

print("Top + words:", top_pos)
print("Top − words:", top_neg)


Top + words: ['delicious', 'great', 'perfect', 'best', 'excellent', 'loves', 'favorite', 'love', 'find', 'tasty', 'with', ')', 'smooth', 'wonderful', 'recommend', 'beef', 'stores', 'glad', 'simply', 'who', 'healthy', 'without', 'free', 'worth', 'max']
Top − words: ['nibs', 'waste', 'horrible', 'were', 'tastes', 'corn', 'away', 'thought', 'raw', 'same', 'stick', 'guess', 'tasted', 'think', 'fine', 'rather', 'worst', 'ok', 't', 'money', 'unfortunately', 'not', 'bad', 'however', 'disappointed']


## 8) Reproducibility

In [ ]:
# Verify the index order files are available
print("First 20 of order(200):", open(os.path.join("/content/data", "200.txt")).read().split(",")[:20])
print("First 20 of order(4000):", open(os.path.join("/content/data", "4000.txt")).read().split(",")[:20])

First 20 of order(200): ['131', '181', '22', '172', '144', '92', '97', '187', '58', '93', '6', '70', '106', '68', '153', '168', '179', '199', '29', '46']
First 20 of order(4000): ['2314', '3128', '1211', '666', '1266', '1627', '2831', '2975', '3163', '2836', '2967', '2585', '1291', '621', '3825', '2794', '61', '195', '1001', '3722']


In [ ]:
T_vals = [1, 3, 5, 10, 15, 25]
L_vals = [1e-3, 3e-3, 1e-2, 3e-2, 1e-1]

# Tune Pegasos T first (fixing lambda), then lambda (fixing T)
tr, va = tune_pegasos_T(L, T_vals, Xtr_sw, train_labels, Xva_sw, val_labels)
best_T = T_vals[int(np.argmax(va))]
print("Tuning T:", dict(zip(T_vals, va)), "| best T:", best_T)

tr, va = tune_pegasos_L(best_T, L_vals, Xtr_sw, train_labels, Xva_sw, val_labels)
best_L = L_vals[int(np.argmax(va))]
print("Tuning lambda:", dict(zip(L_vals, va)), "| best lambda:", best_L)

Tuning T: {1: np.float64(0.708), 3: np.float64(0.742), 5: np.float64(0.744), 10: np.float64(0.804), 15: np.float64(0.776), 25: np.float64(0.78)} | best T: 10
Tuning lambda: {0.001: np.float64(0.784), 0.003: np.float64(0.778), 0.01: np.float64(0.804), 0.03: np.float64(0.746), 0.1: np.float64(0.688)} | best lambda: 0.01
